In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory


#list all the files available in the '/kaggle/input' directory and iterates over the files
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))



In [2]:
#reads the csv file
df=pd.read_csv('C:/Users/Admin/Desktop/python/Hotel_Reviews.csv')

In [3]:
# Selects the first 10,000 rows
df=df.head(10000)
df

,Hotel_Address,Additional_Number_of_Scoring,Review_Date,Average_Score,Hotel_Name,Reviewer_Nationality,Negative_Review,Review_Total_Negative_Word_Counts,Total_Number_of_Reviews,Positive_Review,Review_Total_Positive_Word_Counts,Total_Number_of_Reviews_Reviewer_Has_Given,Reviewer_Score,Tags,days_since_review,lat,lng
0,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,194,8/3/2017,7.7,Hotel Arena,Russia,I am so angry that i made this post available...,397,1403,Only the park outside of the hotel was beauti...,11,7,2.9,"[' Leisure trip ', ' Couple ', ' Duplex Double...",0 days,52.360576,4.915968
1,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,194,8/3/2017,7.7,Hotel Arena,Ireland,No Negative,0,1403,No real complaints the hotel was great great ...,105,7,7.5,"[' Leisure trip ', ' Couple ', ' Duplex Double...",0 days,52.360576,4.915968
2,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,194,7/31/2017,7.7,Hotel Arena,Australia,Rooms are nice but for elderly a bit difficul...,42,1403,Location was good and staff were ok It is cut...,21,9,7.1,"[' Leisure trip ', ' Family with young childre...",3 days,52.360576,4.915968
3,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,194,7/31/2017,7.7,Hotel Arena,United Kingdom,My room was dirty and I was afraid to walk ba...,210,1403,Great location in nice surroundings the bar a...,26,1,3.8,"[' Leisure trip ', ' Solo traveler ', ' Duplex...",3 days,52.360576,4.915968
4,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,194,7/24/2017,7.7,Hotel Arena,New Zealand,You When I booked with your company on line y...,140,1403,Amazing location and building Romantic setting,8,3,6.7,"[' Leisure trip ', ' Couple ', ' Suite ', ' St...",10 days,52.360576,4.915968
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,1 Inverness Terrace Westminster Borough London...,1274,2/14/2017,7.7,Grand Royale London Hyde Park,France,No Negative,0,6539,perfect staff,3,6,9.2,"[' Solo traveler ', ' Club Twin Room ', ' Stay...",170 day,51.510995,-0.186342
9996,1 Inverness Terrace Westminster Borough London...,1274,2/14/2017,7.7,Grand Royale London Hyde Park,United Kingdom,Room size,3,6539,The staff are really friendly,6,1,7.9,"[' Leisure trip ', ' Group ', ' Superior Doubl...",170 day,51.510995,-0.186342
9997,1 Inverness Terrace Westminster Borough London...,1274,2/13/2017,7.7,Grand Royale London Hyde Park,United Kingdom,Room very small Very cramped,7,6539,Friendly,2,2,5.8,"[' Leisure trip ', ' Couple ', ' Superior Doub...",171 day,51.510995,-0.186342
9998,1 Inverness Terrace Westminster Borough London...,1274,2/13/2017,7.7,Grand Royale London Hyde Park,South Africa,No Negative,0,6539,Beds were extra comfy Staff very friendly and...,11,1,9.6,"[' Leisure trip ', ' Family with young childre...",171 day,51.510995,-0.186342


In [4]:
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

In [9]:
#define dictionary for the sentiment scores
afinn_lexicon = {
    'good': 1,
    'great': 2,
    'excellent': 3,
    'bad': -1,
    'terrible': -2,
    'poor': -2,
    'angry':-3
    # Add more words and their sentiment scores as needed
}

In [7]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Admin\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Admin\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [10]:
df['review_text_combined'] = df['Positive_Review'] + " " + df['Negative_Review']
# Preprocess the reviews
stop_words = set(stopwords.words('english'))
def preprocess_text(text):
    tokens = word_tokenize(text.lower())
    tokens = [token for token in tokens if token.isalpha() and token not in stop_words]
    return tokens
df['processed_text'] = df['review_text_combined'].apply(preprocess_text)

# Perform sentiment analysis
def calculate_sentiment_score(tokens):
    sentiment_score = sum(afinn_lexicon.get(token, 0) for token in tokens)
    return sentiment_score
df['sentiment_score'] = df['processed_text'].apply(calculate_sentiment_score)

# Assign sentiment labels based on the score
df['sentiment_label'] = df['sentiment_score'].apply(lambda score: 'positive' if score > 0 else 'negative' if score < 0 else 'neutral')

# Print the results
print(df[['review_text_combined', 'sentiment_score', 'sentiment_label']])

                                   review_text_combined  sentiment_score  \
0      Only the park outside of the hotel was beauti...               -3   
1      No real complaints the hotel was great great ...                9   
2      Location was good and staff were ok It is cut...                1   
3      Great location in nice surroundings the bar a...               -1   
4      Amazing location and building Romantic settin...                0   
...                                                 ...              ...   
9995                          perfect staff No Negative                0   
9996           The staff are really friendly  Room size                0   
9997            Friendly  Room very small Very cramped                 0   
9998   Beds were extra comfy Staff very friendly and...                0   
9999       Breakfast Was horible   Windows Were broken                 0   

     sentiment_label  
0           negative  
1           positive  
2           positi

In [11]:
import sklearn
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Step 2: Create TF-IDF matrix based on review text
tfidf_vectorizer = TfidfVectorizer()
tfidf_matrix = tfidf_vectorizer.fit_transform(df['review_text_combined'])

# Step 3: Calculate similarity matrix based on TF-IDF vectors
similarity_matrix = cosine_similarity(tfidf_matrix)

# Step 4: Define function to get hotel recommendations for a user
def get_recommendations(user_index, num_recommendations=10):
    user_similarities = similarity_matrix[user_index, :]
    sorted_indices = user_similarities.argsort()[::-1]
    recommended_hotels = df['Hotel_Name'].iloc[sorted_indices].values[:num_recommendations]
    return recommended_hotels

# Step 5: Get recommendations for a specific user
user_index = 0  # Index of the user for whom recommendations are needed
recommended_hotels = get_recommendations(user_index)

# Step 6: Print the recommended hotels
print("Recommended hotels for user:", user_index)
for hotel in recommended_hotels:
    print(hotel)

Recommended hotels for user: 0
Hotel Arena
Park Plaza County Hall London
Grand Royale London Hyde Park
Grand Royale London Hyde Park
Grand Royale London Hyde Park
Park Plaza County Hall London
K K Hotel George
Hotel Arena
Grand Royale London Hyde Park
Grand Royale London Hyde Park


In [12]:


# Step 2: Create TF-IDF matrix based on review text
tfidf_vectorizer = TfidfVectorizer()
tfidf_matrix = tfidf_vectorizer.fit_transform(df['review_text_combined'])

# Step 3: Calculate similarity matrix based on TF-IDF vectors
similarity_matrix = cosine_similarity(tfidf_matrix)

# Step 4: Define function to get hotel recommendations for a user
def get_recommendations(user_index, num_recommendations=10):
    user_similarities = similarity_matrix[user_index, :]
    sorted_indices = user_similarities.argsort()[::-1]
    recommended_hotels = []
    recommended_count = 0
    for index in sorted_indices:
        hotel_name = df['Hotel_Name'].iloc[index]
        if hotel_name not in recommended_hotels:
            recommended_hotels.append(hotel_name)
            recommended_count += 1
            if recommended_count >= num_recommendations:
                break
    return recommended_hotels

# Step 5: Get recommendations for a specific user
user_index = 10  # Index of the user for whom recommendations are needed
recommended_hotels = get_recommendations(user_index)

# Step 6: Print the recommended hotels
print("Recommended hotels for user:", user_index)
for hotel in recommended_hotels:
    print(hotel)


Recommended hotels for user: 10
Hotel Arena
The Park Grand London Paddington
Novotel Suites Paris Nord 18 me
Apex Temple Court Hotel
The Principal London
Grand Royale London Hyde Park
Park Plaza County Hall London
Monhotel Lounge SPA
K K Hotel George
Hotel Trianon Rive Gauche


In [13]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Step 1: Data Preprocessing
# Combine positive and negative review texts


# Step 2: Create TF-IDF matrix based on review text
tfidf_vectorizer = TfidfVectorizer()
tfidf_matrix_text = tfidf_vectorizer.fit_transform(df['review_text_combined'])

# Step 3: Create TF-IDF matrix based on tags
tags = df['Tags'].apply(lambda x: ' '.join(eval(x)))  # Convert tags from string to list and join them
tfidf_matrix_tags = tfidf_vectorizer.fit_transform(tags)

# Step 4: Calculate similarity matrix based on TF-IDF vectors (text and tags)
similarity_matrix_text = cosine_similarity(tfidf_matrix_text)
similarity_matrix_tags = cosine_similarity(tfidf_matrix_tags)

# Step 5: Define function to get hotel recommendations for a user
def get_recommendations(user_index, num_recommendations=10):
    user_similarities_text = similarity_matrix_text[user_index, :]
    user_similarities_tags = similarity_matrix_tags[user_index, :]
    combined_similarity = 0.7 * user_similarities_text + 0.3 * user_similarities_tags
    sorted_indices = combined_similarity.argsort()[::-1]
    recommended_hotels = []
    recommended_count = 0
    for index in sorted_indices:
        hotel_name = df['Hotel_Name'].iloc[index]
        if hotel_name not in recommended_hotels:
            recommended_hotels.append(hotel_name)
            recommended_count += 1
            if recommended_count >= num_recommendations:
                break
    return recommended_hotels


# Step 6: Get recommendations for a specific user
user_index = 10  # Index of the user for whom recommendations are needed
recommended_hotels = get_recommendations(user_index)

# Step 7: Print the recommended hotels
print("Recommended hotels for user:", user_index)
for hotel in recommended_hotels:
    print(hotel)


Recommended hotels for user: 10
Hotel Arena
Grand Royale London Hyde Park
The Park Grand London Paddington
The Principal London
Hotel Trianon Rive Gauche
Park Plaza County Hall London
Apex Temple Court Hotel
One Aldwych
K K Hotel George
InterContinental London Park Lane
